In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
#Cargar el Dataset
datapath = 'https://raw.githubusercontent.com/jonathan-elian-toapanta/INTERNETWORKING/main/ESCENARIOS/ScenarioA.csv'
dataframe = pd.read_csv(datapath,low_memory=False, sep=',')
dataframe.head()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,53913,216.58.208.46,80,6,435,0.0,4597.701149,435.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
1,10.0.2.15,53913,216.58.208.46,80,6,259,0.0,7722.007722,259.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
2,10.0.2.15,53913,216.58.208.46,80,6,891,0.0,2244.668911,891.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
3,10.0.2.15,53913,216.58.208.46,80,6,1074,0.0,1862.197393,1074.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
4,10.0.2.15,53913,216.58.208.46,80,6,315,0.0,6349.206349,315.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR


In [3]:
def dfNormalize(df):
    for feature_name in df.columns:
        df.loc[:,feature_name]= pd.to_numeric(df.loc[:,feature_name], errors='coerce').fillna(0)
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()   
        if (max_value - min_value) > 0:
            df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)
        else:
            df.loc[:,feature_name] = (df.loc[:,feature_name]- min_value)    
    return df
dataframe = dataframe.reindex(np.random.permutation(dataframe.index)).copy()
keys = dataframe.keys()
data_to_process = dataframe[keys[4:len(keys) - 1]].copy()
x_normalised = dfNormalize(data_to_process)
print(x_normalised.describe())

<ipython-input-3-29cd7f8bb742>:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,feature_name] = (df.loc[:,feature_name] - min_value) / (max_value - min_value)


           Protocol   Flow Duration   Flow Bytes/s   Flow Packets/s  \
count  67834.000000    67834.000000        67830.0          67828.0   
mean       0.560663        0.299188            0.0              0.0   
std        0.496310        0.406300            0.0              0.0   
min        0.000000        0.000000            0.0              0.0   
25%        0.000000        0.004436            0.0              0.0   
50%        1.000000        0.041086            0.0              0.0   
75%        1.000000        0.732555            0.0              0.0   
max        1.000000        1.000000            0.0              0.0   

        Flow IAT Mean   Flow IAT Std   Flow IAT Max   Flow IAT Min  \
count    67834.000000   67834.000000   67834.000000   67834.000000   
mean         0.031600       0.031363       0.089855       0.019491   
std          0.069971       0.090973       0.173880       0.057865   
min          0.000000       0.000000       0.000000       0.000000   
25%       

In [4]:
change_labels = lambda x: 1 if x == 'nonTOR' else 0
y_normalised = dataframe['label'].apply(change_labels)
X_train, X_test, y_train, y_test = train_test_split(x_normalised,
                                            y_normalised, test_size=0.3, random_state=42)
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)
#Escalar los datos utilizando StandardScaler.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_recall_fscore_support, precision_score, recall_score, f1_score

#Crear el modelo SVM con kernel sigmoid
svm_sigmoid = SVC(kernel='sigmoid', gamma='auto', coef0=6, C=1.0, random_state=42)

#Definir las métricas de evaluación
scoring_sigmoid = {'accuracy': make_scorer(accuracy_score),
                   'precision': make_scorer(precision_score, average='micro'),
                   'recall': make_scorer(recall_score, average='micro'),
                   'f1_score': make_scorer(f1_score, average='micro')}

#Realizar la validación cruzada con 5 folds
cv_results_sigmoid = cross_validate(svm_sigmoid, X_train_scaled, y_train, cv=5, scoring=scoring_sigmoid)

#Imprimir los resultados de la validación cruzada
print('Accuracy:', cv_results_sigmoid['test_accuracy'].mean()*100)
print('Precision:', cv_results_sigmoid['test_precision'].mean()*100)
print('Recall:', cv_results_sigmoid['test_recall'].mean()*100)
print('F1-score:', cv_results_sigmoid['test_f1_score'].mean()*100)

Accuracy: 88.18945751414071
Precision: 88.18945751414071
Recall: 88.18945751414071
F1-score: 88.18945751414071


# **Escenario A KERNEL=LINEAR**






In [5]:
#Crear el modelo SVM con kernel lineal
svm_linear = SVC(kernel='linear', C=0.00001, random_state=42)

#Definir las métricas de evaluación
scoring_linear = {'accuracy': make_scorer(accuracy_score),
                  'precision': make_scorer(precision_score, average='micro'),
                  'recall': make_scorer(recall_score, average='micro'),
                  'f1_score': make_scorer(f1_score, average='micro')}

#Realizar la validación cruzada con 5 folds
cv_results_linear = cross_validate(svm_linear, X_train_scaled, y_train, cv=5, scoring=scoring_linear)

#Imprimir los resultados de la validación cruzada
print('Accuracy:', cv_results_linear['test_accuracy'].mean()*100)
print('Precision:', cv_results_linear['test_precision'].mean()*100)
print('Recall:', cv_results_linear['test_recall'].mean()*100)
print('F1-score:', cv_results_linear['test_f1_score'].mean()*100)

Accuracy: 88.18945751414071
Precision: 88.18945751414071
Recall: 88.18945751414071
F1-score: 88.18945751414071


# **Escenario A KERNEL= RBF**

In [6]:
#Crear el modelo SVM con kernel rbf
svm_rbf = SVC(kernel='rbf', gamma=0.1, C=0.0001, random_state=42)

#Definir las métricas de evaluación
scoring_rbf = {'accuracy': make_scorer(accuracy_score),
               'precision': make_scorer(precision_score, average='micro'),
               'recall': make_scorer(recall_score, average='micro'),
               'f1_score': make_scorer(f1_score, average='micro')}

#Realizar la validación cruzada con 5 folds
cv_results_rbf = cross_validate(svm_rbf, X_train_scaled, y_train, cv=5, scoring=scoring_rbf)

#Imprimir los resultados de la validación cruzada
print('Accuracy:', cv_results_rbf['test_accuracy'].mean()*100)
print('Precision:', cv_results_rbf['test_precision'].mean()*100)
print('Recall:', cv_results_rbf['test_recall'].mean()*100)
print('F1-score:', cv_results_rbf['test_f1_score'].mean()*100)

Accuracy: 88.18945751414071
Precision: 88.18945751414071
Recall: 88.18945751414071
F1-score: 88.18945751414071


# **Escenario A KERNEL= POLINOMYAL**

In [7]:
#Crear el modelo SVM con kernel polinómico
svm_poly = SVC(kernel='poly', degree=1, coef0=1, C=0.0001, random_state=42)

#Definir las métricas de evaluación
scoring_poly = {'accuracy': make_scorer(accuracy_score),
                'precision': make_scorer(precision_score, average='micro'),
                'recall': make_scorer(recall_score, average='micro'),
                'f1_score': make_scorer(f1_score, average='micro')}

#Realizar la validación cruzada con 5 folds
cv_results_poly = cross_validate(svm_poly, X_train_scaled, y_train, cv=5, scoring=scoring_poly)

#Imprimir los resultados de la validación cruzada
print('Accuracy:', cv_results_poly['test_accuracy'].mean()*100)
print('Precision:', cv_results_poly['test_precision'].mean()*100)
print('Recall:', cv_results_poly['test_recall'].mean()*100)
print('F1-score:', cv_results_poly['test_f1_score'].mean()*100)

Accuracy: 88.18945751414071
Precision: 88.18945751414071
Recall: 88.18945751414071
F1-score: 88.18945751414071
